In [1]:
import numpy as np
import torch
import sys
sys.path.append('../')
from voting_games.werewolf_env_v0 import plurality_env, Roles, Phase
import random
import copy
from tqdm import tqdm
from tabulate import tabulate

/root/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Plurality Voting

Blurb on plurality voting goes here...

We want to see how an static agents vs static wolves fare, before training our PPO agents to hopefully learn to do better

## Pre-Training Baselines

To properly asses our agents, we need baselines. For this purpose we have totally random villagers and semi-random villagers that will only vote for agents that are currently alive.

As for wolves, we have the following behaviors:
- wolves that coordinate and each target one villager
- random wolves that do whatever
- revenge wolves that coordinate and target a random villager that targetted a wolf

In [5]:
def revenge_coordinated_wolf(env, agent, action = None):
    villagers_remaining = set(env.world_state["villagers"]) & set(env.world_state['alive'])
    wolves_remaining = set(env.world_state["werewolves"]) & set(env.world_state['alive'])

    # who tried to vote out a wolf last time?
    # TODO : just get this from the observations

    prev_votes = env.history[-1]['votes']
    villagers_targetting_wolves = [player for player in prev_votes if f'player_{prev_votes[player]}' in wolves_remaining]
    # target = random.choice(list(villagers_remaining))
    # pick 
    if action != None:
        return action
    
    if len(villagers_targetting_wolves) == 0:
        return int(random.choice(list(villagers_remaining)).split("_")[-1])
    
    return int(random.choice(list(villagers_targetting_wolves)).split("_")[-1])

def revenge_wolf(env, agent, action = None):
    villagers_remaining = set(env.world_state["villagers"]) & set(env.world_state['alive'])

    prev_votes = env.history[-1]['votes']
    villagers_targetting_you = [player for player in prev_votes if f'player_{prev_votes[player]}' == agent]
    if len(villagers_targetting_you) > 0:
        return int(random.choice(list(villagers_targetting_you)).split("_")[-1])
    
    return int(random.choice(list(villagers_remaining)).split("_")[-1])

def random_single_target_villager(env, agent, action=None):
    targets = set(env.world_state["alive"]) - set([agent])
    return int(random.choice(list(targets)).split("_")[-1])

def random_single_target_coordinated_villager(env, agent, action=None):
    targets = set(env.world_state["alive"]) - set([agent])
    return action if action != None else int(random.choice(list(targets)).split("_")[-1])

# random_coordinated_wolf(env)
def random_agent_action(env, agent, action=None):
   return env.action_space(agent).sample()

def random_coordinated_single_wolf(env, agent, action=None):
    villagers_remaining = set(env.world_state["villagers"]) & set(env.world_state['alive'])
    return action if action != None else int(random.choice(list(villagers_remaining)).split("_")[-1])

In [6]:
def play_static_wolf_game(env, wolf_policy, villager_agent, num_times=100):

    villager_wins = 0
    game_replays = []
    for _ in range(num_times):
        observations, rewards, terminations, truncations, infos = env.reset()
        
        wolf_action = None
        villager_action = None
        while env.agents:
            actions = {}

            villagers = set(env.agents) & set(env.world_state["villagers"])
            wolves = set(env.agents) & set(env.world_state["werewolves"])

            # villager steps
            for villager in villagers:
                villager_action = villager_agent(env, villager, action=villager_action)
                actions[villager] = villager_action

            # wolf steps
            phase = env.world_state['phase']
            for wolf in wolves:
                wolf_action = wolf_policy(env, wolf, action=wolf_action)
                actions[wolf] = wolf_action
        
            observations, rewards, terminations, truncations, infos = env.step(actions)

            if env.world_state['phase'] == Phase.NIGHT:
                wolf_action = None
            
            if env.world_state['phase'] == Phase.ACCUSATION and phase == Phase.NIGHT:
                wolf_action = None

            # If we care to use villager action
            villager_action = None

        winner = env.world_state['winners']
        if winner == Roles.VILLAGER:
            villager_wins += 1

        game_replays.append(copy.deepcopy(env.history))

    return villager_wins, game_replays

In [9]:
env = plurality_env(num_agents=10, werewolves=2, num_accusations=1)
env.reset()
print(f'10 players, with 2 wolves\n')

revenge_coordinated_wolves = []
revenge_coordinated_wolves.append(play_static_wolf_game(env, revenge_coordinated_wolf, random_single_target_villager, num_times=1000)[0]/1000.0)
revenge_coordinated_wolves.append(play_static_wolf_game(env, revenge_coordinated_wolf, random_agent_action, num_times=1000)[0]/1000.0)
revenge_coordinated_wolves.append(play_static_wolf_game(env, revenge_coordinated_wolf, random_single_target_coordinated_villager, num_times=1000)[0]/1000.0)

revenge_wolves = []
revenge_wolves.append(play_static_wolf_game(env, revenge_wolf, random_single_target_villager, num_times=1000)[0]/1000.0)
revenge_wolves.append(play_static_wolf_game(env, revenge_wolf, random_agent_action, num_times=1000)[0]/1000.0)
revenge_wolves.append(play_static_wolf_game(env, revenge_wolf, random_single_target_coordinated_villager, num_times=1000)[0]/1000.0)

coordinated_wolves = []
coordinated_wolves.append(play_static_wolf_game(env, random_coordinated_single_wolf, random_single_target_villager, num_times=1000)[0]/1000.0)
coordinated_wolves.append(play_static_wolf_game(env, random_coordinated_single_wolf, random_agent_action, num_times=1000)[0]/1000.0)
coordinated_wolves.append(play_static_wolf_game(env, random_coordinated_single_wolf, random_single_target_coordinated_villager, num_times=1000)[0]/1000.0)

random_wolves = []
random_wolves.append(play_static_wolf_game(env, random_agent_action, random_single_target_villager, num_times=1000)[0]/1000.0)
random_wolves.append(play_static_wolf_game(env, random_agent_action, random_agent_action, num_times=1000)[0]/1000.0)
random_wolves.append(play_static_wolf_game(env, random_agent_action, random_single_target_coordinated_villager, num_times=1000)[0]/1000.0)

print(tabulate([['Coordinated Wolves', *coordinated_wolves], ['Random Wolves', *random_wolves], ['Coorindated revenge wolves', *revenge_coordinated_wolves], ['Revenge wolves', *revenge_wolves]], 
               headers=["Wolf Strategy", "Semi-Smart Villager", "Totally Random Villager", "Coordinated Random Villagers"]))

print("\n")
env = plurality_env(num_agents=15, werewolves=3, num_accusations=1)
env.reset()
print(f'15 players, with 3 wolves\n')

revenge_coordinated_wolves = []
revenge_coordinated_wolves.append(play_static_wolf_game(env, revenge_coordinated_wolf, random_single_target_villager, num_times=1000)[0]/1000.0)
revenge_coordinated_wolves.append(play_static_wolf_game(env, revenge_coordinated_wolf, random_agent_action, num_times=1000)[0]/1000.0)
revenge_coordinated_wolves.append(play_static_wolf_game(env, revenge_coordinated_wolf, random_single_target_coordinated_villager, num_times=1000)[0]/1000.0)

revenge_wolves = []
revenge_wolves.append(play_static_wolf_game(env, revenge_wolf, random_single_target_villager, num_times=1000)[0]/1000.0)
revenge_wolves.append(play_static_wolf_game(env, revenge_wolf, random_agent_action, num_times=1000)[0]/1000.0)
revenge_wolves.append(play_static_wolf_game(env, revenge_wolf, random_single_target_coordinated_villager, num_times=1000)[0]/1000.0)

coordinated_wolves = []
coordinated_wolves.append(play_static_wolf_game(env, random_coordinated_single_wolf, random_single_target_villager, num_times=1000)[0]/1000.0)
coordinated_wolves.append(play_static_wolf_game(env, random_coordinated_single_wolf, random_agent_action, num_times=1000)[0]/1000.0)
coordinated_wolves.append(play_static_wolf_game(env, random_coordinated_single_wolf, random_single_target_coordinated_villager, num_times=1000)[0]/1000.0)

random_wolves = []
random_wolves.append(play_static_wolf_game(env, random_agent_action, random_single_target_villager, num_times=1000)[0]/1000.0)
random_wolves.append(play_static_wolf_game(env, random_agent_action, random_agent_action, num_times=1000)[0]/1000.0)
random_wolves.append(play_static_wolf_game(env, random_agent_action, random_single_target_coordinated_villager, num_times=1000)[0]/1000.0)

print(tabulate([['Coordinated Wolves', *coordinated_wolves], ['Random Wolves', *random_wolves], ['Coorindated revenge wolves', *revenge_coordinated_wolves], ['Revenge wolves', *revenge_wolves]], 
               headers=["Wolf Strategy", "Semi-Smart Villager", "Totally Random Villager", "Coordinated Random Villagers"]))

10 players, with 2 wolves

Wolf Strategy                 Semi-Smart Villager    Totally Random Villager    Coordinated Random Villagers
--------------------------  ---------------------  -------------------------  ------------------------------
Coordinated Wolves                          0.125                      0.042                           0.275
Random Wolves                               0.671                      0.591                           0.665
Coorindated revenge wolves                  0.22                       0.086                           0.308
Revenge wolves                              0.225                      0.07                            0.252


15 players, with 3 wolves

Wolf Strategy                 Semi-Smart Villager    Totally Random Villager    Coordinated Random Villagers
--------------------------  ---------------------  -------------------------  ------------------------------
Coordinated Wolves                          0.027                      0